In [3]:
u.user <- read.csv(file = 'u.user.csv', sep='|', header=T)
u.item <- read.csv(file = 'u.item.csv', sep='|', header=T)
u.data <- read.csv(file = 'u.data.csv', sep='|', header=T)
colMeans.sparse <- function(m) {colSums(m)/colSums(m>0)}
rowMeans.sparse <- function(m) {rowSums(m)/rowSums(m>0)}
means.sparse <- function(v) {sum(v)/sum(v>0)}

In [4]:
library(Matrix)
library(Hmisc)

Loading required package: lattice
Loading required package: survival
Loading required package: Formula
Loading required package: ggplot2

Attaching package: 'Hmisc'

The following objects are masked from 'package:base':

    format.pval, units



In [5]:
head(u.data)

user.id,item.id,rating,timestamp
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596
298,474,4,884182806


In [6]:
head(u.item)

movie.id,movie.title,release.date,video.release.date,IMDb.URL,unknown,Action,Adventure,Animation,Children.s,...,Fantasy,Film.Noir,Horror,Musical,Mystery,Romance,Sci.Fi,Thriller,War,Western
1,Toy Story (1995),01-Jan-1995,NA,http://us.imdb.com/M/title-exact?Toy%20Story%20(1995),0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NA,http://us.imdb.com/M/title-exact?GoldenEye%20(1995),0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NA,http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NA,http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995),0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NA,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6,Shanghai Triad (Yao a yao yao dao waipo qiao) (1995),01-Jan-1995,NA,http://us.imdb.com/Title?Yao+a+yao+yao+dao+waipo+qiao+(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
## Question 1
# on associe les deux bases de données
v.u <- merge(u.data, u.user, by.x='user.id', by.y='id')

In [8]:
head(v.u)

user.id,item.id,rating,timestamp,age,gender,job,zip
1,1,5,874965758,24,M,technician,85711
1,23,4,875072895,24,M,technician,85711
1,223,5,876892918,24,M,technician,85711
1,171,5,889751711,24,M,technician,85711
1,16,5,878543541,24,M,technician,85711
1,73,3,876892774,24,M,technician,85711


In [9]:
# on calcule la moyenne des votes selon la liste des jobs
moyenne.job <- aggregate(v.u$rating, FUN=mean, by=list(v.u$job))
colnames(moyenne.job) <- c("job", "rating average")
moyenne.job

job,rating average
administrator,3.635646
artist,3.653380
doctor,3.688889
educator,3.670621
engineer,3.541407
entertainment,3.441050
executive,3.349104
healthcare,2.896220
homemaker,3.301003
lawyer,3.735316


In [10]:
# on calcule la moyenne des votes selon la liste des ages
moyenne.age <- aggregate(v.u$rating, list(cut (v.u$age, 5*(0:20))),mean)
colnames(moyenne.age) <- c("age", "average rating")
moyenne.age

age,rating average
"(5,10]",3.608108
"(10,15]",3.367932
"(15,20]",3.567306
"(20,25]",3.439154
"(25,30]",3.447106
"(30,35]",3.574980
"(35,40]",3.571040
"(40,45]",3.581070
"(45,50]",3.567870
"(50,55]",3.719114


In [11]:
# mettre en haut
# Cosinus entre un vecteur v et chaque colonne de la matrice m
cosinus.vm <- function(v,m) { 
    n <- sqrt(colSums(m^2)); 
    (v %*% m)/(n * sqrt(sum(v^2)))
}

# Trouve les indexes des premieres 'n' valeurs maximales d'une matrice
max.nindex <- function(m, n=5) {
    i <- order(m, decreasing=TRUE)
    return(i[1:n])
}
# Trouve les indexes des premieres 'n' valeurs minimales d'une matrice
min.nindex <- function(m, n=5) {
    i <- order(m)
    return(i[1:n])
}

In [12]:
# mettre en haut
m.sparse <- sparseMatrix(u.data[,1],u.data[,2],x=u.data[,3])
m.sparse.t <- t(m.sparse)
m <- as.matrix(m.sparse)
m[m==0] <- NA
rownames(m) <- paste('u', 1:nrow(m), sep='')
colnames(m) <- paste('i', 1:ncol(m), sep='')

In [65]:
# Question 2
# on trouve le ID du film Star Trek V: The Final Frontier (1989)
ST_id <- u.item$movie.id[u.item$movie.title=="Star Trek V: The Final Frontier (1989)"]
# le movie.id est 450
ST_id

[1] 450

In [81]:
# on calcule le cosinus entre les films et star trek V
ST.cos <- cosinus.vm(m.sparse[,ST_id], m.sparse)
# on garde les 11 films les plus rapprochés moins star trek V
closestCos <- max.nindex(as.matrix(ST.cos), 11)[-1]
closestCos <- u.item$movie.title[closestCos]
# on retrouve les 10 films les plus proches
closestCos
# on remarque que la plupart des films est en relation avec Star Trek 
# (Star Trek et Stargate) 8 films de Star Trek sont présents et parmis les 7
# n'incluant pas The final Frontier 6 sont présents

[1] Star Trek: The Motion Picture (1979)         
 [2] Star Trek VI: The Undiscovered Country (1991)
 [3] Star Trek III: The Search for Spock (1984)   
 [4] Star Trek IV: The Voyage Home (1986)         
 [5] Star Trek: The Wrath of Khan (1982)          
 [6] Stargate (1994)                              
 [7] Star Trek: Generations (1994)                
 [8] Die Hard 2 (1990)                            
 [9] Escape from New York (1981)                  
[10] Conan the Barbarian (1981)                   
1664 Levels: 'Til There Was You (1997) 1-900 (1994) ... Zeus and Roxanne (1997)

In [74]:

## retourne la correlation d une matrice creuse et d un vecteur v
cor.vm <- function(v,m) {
    v_a <- means.sparse(v);
    v_i <- colMeans.sparse(m);
    m.center <- t(t(m) - v_i)*(m>0);
    v.center <- (v-v_a)*(v>0);
    n <- colSums(m.center^2);
    (v.center %*% m.center)/sqrt(n * sum(v.center^2))
}
# on calcule la correlation entre les films et star trek V
# on ne prend pas la colonne ST_id?????
ST.cor <- cor.vm(m.sparse[,ST_id], m.sparse)
#ST.cor <- cor((m[,ST_id]),  m[,-ST_id] , use = "pairwise.complete.obs")

# on garde les 11 films les plus rapprochés moins star trek V
closestCor <- max.nindex(as.matrix(ST.cor), 11)[-1]
closestCor <- u.item$movie.title[c(closestCor)]
# on retrouve les 10 films les plus proches
closestCor

# on retrouve encore une fois plusieurs films de Star Trek, mais seulement 3
# dans cette situation. Il y a Robocop avec lequel on pourrait voir une relation.
# les autres recommandations sont quant à elles un peu plus étrange.




[1] Star Trek: The Motion Picture (1979)      
 [2] Star Trek III: The Search for Spock (1984)
 [3] Robocop 3 (1993)                          
 [4] Just Cause (1995)                         
 [5] Three Musketeers, The (1993)              
 [6] Star Trek: Generations (1994)             
 [7] Money Train (1995)                        
 [8] House Party 3 (1994)                      
 [9] Mr. Magoo (1997)                          
[10] S.F.W. (1994)                             
1664 Levels: 'Til There Was You (1997) 1-900 (1994) ... Zeus and Roxanne (1997)

In [75]:
# on répète la même procédure mais avec un nombre de votes communs minimum
# entre les films

# on calcule le nombre de votes communs des films avec star trek V
votes.communs <- (colSums((m.sparse[,ST_id] * m.sparse) > 0))
ST.cor[votes.communs<5] <- -Inf
closestCor <- max.nindex(as.matrix(ST.cor), 11)[-1]
closestCor <- u.item$movie.title[c(closestCor)]
# on retrouve les 10 films les plus proches
closestCor

# En ajustant le nombre de votes communs minimum, on peut enlever des
# recommandation qui faisait un peu moins de sens, par exemple House Party 3.
# De plus, avec cela on obtient une recommendation de Star Trek de plus.

[1] Star Trek: The Motion Picture (1979)         
 [2] Star Trek III: The Search for Spock (1984)   
 [3] Robocop 3 (1993)                             
 [4] Just Cause (1995)                            
 [5] Three Musketeers, The (1993)                 
 [6] Star Trek: Generations (1994)                
 [7] Money Train (1995)                           
 [8] Mr. Magoo (1997)                             
 [9] Star Trek VI: The Undiscovered Country (1991)
[10] Disclosure (1994)                            
1664 Levels: 'Til There Was You (1997) 1-900 (1994) ... Zeus and Roxanne (1997)

In [18]:
#essaie
temp <-(((m.sparse[,450] - m.sparse))^2)*((m.sparse[,450]>0- (m.sparse>0)))^2
new.dist <- sqrt(colSums((temp>0)))
v_i <- colMeans.sparse(m.sparse)



In [19]:
## Question 3

voisins <- 21

# on centre la matrice m selon les utilisateurs
#### dire pourquoi on la centre pour enlever le biais des utilisateur
u.mean <-rowMeans.sparse(m.sparse)
m.center <- as.matrix((m.sparse-u.mean)*(m.sparse>0))
# on remplace les 0 par NA
m.center[m.center ==0] <- NA
# on calcule la distance eucledienne entre les films
distance.na.450.center <- sqrt(colSums((m.center-m.center[,ST_id])^2, na.rm=T)) # ignore les valeurs manquantes

# on assigne la valeur au film qui ont moins de 10 votes communs avec star trek V
distance.na.450.center[votes.communs<10] <- Inf


In [20]:
# on retrouve les 21 films les plus rapproches et on enleve star trek V
closest.euc <- min.nindex(distance.na.450.center, voisins)[-1]
closestEuc <- u.item$movie.title[closest.euc]
closestEuc

[1] Jade (1995)                   Spy Hard (1996)              
 [3] Just Cause (1995)             Diabolique (1996)            
 [5] Walk in the Clouds, A (1995)  Nick of Time (1995)          
 [7] Milk Money (1994)             Disclosure (1994)            
 [9] Private Benjamin (1980)       Flintstones, The (1994)      
[11] Money Train (1995)            Little Women (1994)          
[13] Phantom, The (1996)           Days of Thunder (1990)       
[15] Batman & Robin (1997)         Super Mario Bros. (1993)     
[17] Amityville Horror, The (1979) Nine Months (1995)           
[19] Chain Reaction (1996)         Johnny Mnemonic (1995)       
1664 Levels: 'Til There Was You (1997) 1-900 (1994) ... Zeus and Roxanne (1997)

In [21]:
# on calcule le cosinus entre les films choisis et star trek V
wcos.st <- cosinus.vm(m.sparse[,ST_id], m.sparse[,closest.euc])
wcos.st

1 x 20 Matrix of class "dgeMatrix"
          [,1]      [,2]      [,3]      [,4]      [,5]      [,6]      [,7]
[1,] 0.1616171 0.1387985 0.2560321 0.1910393 0.1342664 0.2042237 0.1817954
          [,8]      [,9]     [,10]     [,11]    [,12]     [,13]     [,14]
[1,] 0.2514799 0.1891286 0.1098201 0.2803367 0.124555 0.1823567 0.1231804
         [,15]     [,16]     [,17]     [,18]     [,19]     [,20]
[1,] 0.1616171 0.2158619 0.1635889 0.2084909 0.1468426 0.2121889

In [80]:
# les 20 films les plus communs
idx<- closest.euc
# on transpose m
m.sparse.t <- t(m.sparse)
# la range de l item de star trek V
v.item <- m.sparse.t[ST_id,]
# la moyenne de l item de star trek V
v_1 <-means.sparse(v.item);
# retourne la moyenne pour chacun des 20 films voisisns
v_i <- apply(m.sparse.t[idx,], 1, FUN=means.sparse)


x<- as.matrix((m.sparse.t[idx,]))
new_x <-lapply(seq_len(ncol(x)), function(i) x[,i])
new_y <-as.vector(lapply(seq_len(ncol(x)), function(i) v_i))
new_w <-as.vector(lapply(seq_len(ncol(x)), function(i) as.matrix(wcos.st)))

try <- function(x,y,w) {x.0<-x>0;x.temp <- (x-y)*x.0;num <-  w %*%x.temp;denum <- sum(abs(t(w))*x.0);num/denum}


c_prod <- mapply(try ,new_x, new_y, new_w)
prediction <- v_1+c_prod
prediction[v.item==0]

[1]  2.819618920  3.194042473  0.396825397  2.549367770  2.627390931
  [6]  3.071108781  2.940035273  2.838627640  0.870640123  1.484116526
 [11]  1.063307572  1.263668535  3.292950350          NaN  2.546248736
 [16]          NaN  0.807376030  2.985035454  2.794091999  2.224370595
 [21]  3.296125428  1.841158157  3.113532419          NaN          NaN
 [26]          NaN  2.212614871          NaN          NaN          NaN
 [31]          NaN          NaN  2.249905582          NaN  3.396825397
 [36]          NaN  2.464867080  2.396000678  3.113532419  3.113532419
 [41]          NaN  3.150619119  2.056002431          NaN          NaN
 [46]  2.525147060          NaN  2.113532419          NaN  1.959067337
 [51]  2.607363553  3.453075949  2.941895110          NaN  2.044742760
 [56]          NaN  2.990782362  2.936825397  2.113532419          NaN
 [61]  2.559648365          NaN          NaN  2.763426356  2.790770644
 [66]          NaN          NaN  3.109635419  1.705370161          NaN
 [71]  2.385631367  2.940035273  2.385631367  1.245749986  2.277779883
 [76]          NaN  2.282172360          NaN  2.279070470          NaN
 [81]  3.679764842  2.678260994  2.766229234  2.385631367  2.964162590
 [86]  2.921060116  3.362461136          NaN  0.392028508          NaN
 [91]          NaN  2.113532419  1.113532419          NaN  2.385631367
 [96]          NaN  2.559648365  1.593874489  2.274914297          NaN
[101]          NaN  2.113532419  2.456478963  3.028563897  2.345173577
[106]  3.549367770  4.130421793  3.425207323  2.113532419  2.626117816
[111]  2.590056461  2.555377394  3.549367770  2.593726518  4.396825397
[116]  2.807263292          NaN  1.507569955  1.864798675          NaN
[121]          NaN  2.362461136  2.858262758  3.362461136  2.940035273
[126]  3.113532419          NaN  0.342393942          NaN          NaN
[131]  2.220505853          NaN          NaN -0.886467581  1.396825397
[136]  3.113532419  2.442326254  2.709376972  1.113532419  3.382377652
[141]          NaN  1.171026908  3.113532419  3.035935941  2.995075836
[146]  1.038721208  2.549367770          NaN          NaN  2.113532419
[151]          NaN          NaN  1.385631367  1.584349332          NaN
[156]          NaN  2.147640678          NaN  3.549367770  2.881593818
[161]  2.230158730  2.726500706  2.623511812          NaN  4.060618500
[166]  0.586728790          NaN  2.870854739  2.896800969          NaN
[171]  3.385631367  2.600927932  2.618896088          NaN          NaN
[176]  2.113532419  3.113532419  2.854836566          NaN  2.892311980
[181]  2.988236898          NaN  3.253481138  1.849897491          NaN
[186]          NaN          NaN          NaN  1.396825397  2.141968100
[191]          NaN  3.113532419  3.119111319  1.507569955  0.113532419
[196]  3.079051439  2.001209118  3.142046813  1.583189303  3.362461136
[201]          NaN          NaN          NaN  1.549367770  2.198238644
[206]          NaN  1.385631367  2.113532419  1.385631367          NaN
[211]  1.755693569  1.113532419  1.575347466  3.114161570  2.233081792
[216]  3.125424368  2.035296261  3.097247468          NaN  3.087202807
[221]          NaN          NaN  3.385631367  1.858262758  2.765328108
[226]          NaN  1.262845894          NaN  3.161060691  2.732529498
[231]  2.434000882  2.886122239  3.385631367  3.064215363  2.230533946
[236]  1.336940868  2.339898369  2.629844178          NaN  2.571582678
[241]          NaN          NaN  2.155436877  2.470081286  3.399070886
[246]  1.111111111  3.385631367  1.911916441  3.548198356  2.960735505
[251]  2.875698678          NaN  3.077413757  3.111111111          NaN
[256]  2.397750219          NaN          NaN          NaN          NaN
[261]  2.362461136  2.488322062  2.907403217  2.979339451          NaN
[266]  2.387896809  3.162378811  2.476929798  2.607525577  2.959713720
[271]  2.466676429  2.288161765  3.030202034          NaN  2.324689107
[276]          NaN  2.385631367  2.141410295  2.385631367  2.183657362
[281]          NaN  3.3856

In [23]:
## Question 4
# on calcule l'erreur quadratique moyenne en comparant avec les valeurs observés
x.real <-m.sparse.t[ST_id,]
x.real.0 <- x.real>0
sqrt(sum((prediction-x.real)^2*(x.real.0), na.rm=TRUE)/sum(x.real.0))



[1] 0.7255201

In [24]:
## Question 5
# creation de l utilisateur artificiel
new.user <- rep(0, 1682)
new.user.na <- new.user
new.user.na[new.user.na == 0] <- NA 

In [25]:
# film de star wars avec note de 1
new.user[172] <- 1
new.user.na[172] <- 1
new.user[181] <- 1
new.user.na[181] <- 1
# films de star treck avec note de 5
indices.star.trek <- grep("trek", as.character(u.item$movie.title), ignore.case=T)
new.user[indices.star.trek] <- 5
new.user.na[indices.star.trek] <- 5
total.idx <- c(indices.star.trek,172,181)

In [58]:
voisins <- 20
# centrer selon lutilisateur les notes des films quil a donne
# on calcule la moyenne et centre pour enlever les biais des utilisateurs
u.mean <-rowMeans.sparse(m.sparse)
m.center <- as.matrix((m.sparse-u.mean)*(m.sparse>0))
# centrer selon lutilisateur les notes des films quil a donne
new.user.na.center <- new.user.na - mean(new.user.na, na.rm=TRUE)
m.center.sparse <- m.center
m.center.sparse.t <- t(m.center.sparse)
m.center[m.center==0] <- NA
distance.na.450.center <- sqrt(colSums((t(m.center)-new.user.na.center)^2, na.rm=T)) # ignore les valeurs manquantes
votes.communs <- (colSums((m.sparse.t*new.user) > 0))

distance.na.450.center[votes.communs<6]<-Inf

In [59]:
closest.euc <- min.nindex(distance.na.450.center, voisins)
closestEuc <- u.user$id[closest.euc]
closestEuc
closest.euc

[1] 127  11 551 927 183 702 337 806 788 200 374 897 932 561 275 380 279 498  60
[20] 483

[1] 127  11 551 927 183 702 337 806 788 200 374 897 932 561 275 380 279 498  60
[20] 483

In [60]:
# le poids des voisins
wcos.st <- cosinus.vm(new.user, m.sparse.t[,closest.euc])
new.user[c(indices.star.trek,172,181)]
temp <- t(m.sparse[closest.euc,c(indices.star.trek,172,181)])
temp
wcos.st

temp.sum <- t(colSums(temp>0))
correction <- lapply(seq_len(ncol(temp.sum)), function(i) max(temp.sum[i],5)/5)
correction <- unlist(correction)
wcos.st <- correction*wcos.st

[1] 5 5 5 5 5 5 5 5 1 1

10 x 20 sparse Matrix of class "dgCMatrix"
                                             
 [1,] 5 3 5 5 4 5 5 4 3 5 4 4 4 3 4 3 1 3 4 3
 [2,] 4 3 5 2 4 4 5 2 3 5 4 3 . . 3 . 4 . 4 3
 [3,] 5 3 5 5 4 5 5 4 3 5 5 4 4 3 4 3 4 2 4 5
 [4,] 5 4 5 3 3 4 3 . 3 5 5 . 4 3 3 3 4 2 4 3
 [5,] 5 4 5 5 5 4 5 4 3 5 5 4 4 3 3 . 4 . 4 5
 [6,] 5 . 3 5 4 4 4 . 3 5 . . 4 2 3 . 4 . . 3
 [7,] 4 3 . 4 2 3 4 . . 5 4 . . . 3 3 3 3 . 3
 [8,] 5 . . . 3 1 2 . . . 4 . . . 3 . 4 . . 4
 [9,] . . 2 . . . . 3 3 5 3 4 5 2 . 3 2 3 4 .
[10,] . . 2 . 2 . 2 2 . . 3 3 . 3 4 3 3 2 4 4

1 x 20 Matrix of class "dgeMatrix"
          [,1]     [,2]      [,3]      [,4]      [,5]      [,6]      [,7]
[1,] 0.6351496 0.145794 0.1374697 0.2410708 0.4254494 0.6596227 0.4921567
          [,8]      [,9]     [,10]     [,11]     [,12]     [,13]      [,14]
[1,] 0.1183525 0.1197865 0.2090281 0.1923859 0.1038749 0.1164911 0.08758551
       [,15]     [,16]     [,17]      [,18]     [,19]     [,20]
[1,] 0.30493 0.1096981 0.1407229 0.09282439 0.1274287 0.4279913

In [61]:
idx<- closest.euc
# on transpose m

v.item <- new.user
v_1 <-means.sparse(v.item);
# return means for each item
v_i <- apply(m.sparse[idx,], 1, FUN=means.sparse)
# I_ij <- apply(c(m.sparse.t[idx,],v_i,v_1), 1, FUN=means.sparse)

x<- as.matrix((m.sparse[idx,]))
new_x <-lapply(seq_len(ncol(x)), function(i) x[,i])
new_y <-as.vector(lapply(seq_len(ncol(x)), function(i) v_i))
new_w <-as.vector(lapply(seq_len(ncol(x)), function(i) as.matrix(wcos.st)))

try <- function(x,y,w) {x.0<-x>0;x.temp <- (x-y)*x.0;num <-  sum(w*x.temp);denum <- sum(abs(w)*x.0);num/denum}


c_prod <- mapply(try ,new_x, new_y, new_w)
prediction <- v_1+c_prod
prediction[total.idx]
recomendation <- max.nindex(prediction*(colSums(m.sparse[idx,]>0)>3), 10)
total.idx
prediction[recomendation]

[1] 4.970554 4.596225 5.304084 4.520860 5.274442 4.757495 4.190720 3.953147
 [9] 3.880209 3.652659

[1] 222 227 228 229 230 380 449 450 172 181

[1] 5.415754 5.392932 5.328440 5.305594 5.304084 5.274442 5.264440 5.228115
 [9] 5.212419 5.202383

In [62]:
recomendation
closestEuc <- u.item$movie.title[recomendation]
closestEuc

[1] 192 485 659 498 228 230 465 474 187 223

[1] Raging Bull (1980)                                                         
 [2] My Fair Lady (1964)                                                        
 [3] Arsenic and Old Lace (1944)                                                
 [4] African Queen, The (1951)                                                  
 [5] Star Trek: The Wrath of Khan (1982)                                        
 [6] Star Trek IV: The Voyage Home (1986)                                       
 [7] Jungle Book, The (1994)                                                    
 [8] Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)
 [9] Godfather: Part II, The (1974)                                             
[10] Sling Blade (1996)                                                         
1664 Levels: 'Til There Was You (1997) 1-900 (1994) ... Zeus and Roxanne (1997)

In [305]:
dim(m.center)

[1]  943 1682

In [173]:
sum(!is.na(new.user - t(m.center)))

[1] 2299

In [62]:
((1-1.67)*(1-2.33)+(1-1.67)*(2-2.33)+(3-1.67)*(4-2.33))

[1] 3.3333

In [61]:
sqrt(((1-1.67)^2+(1-1.67)^2+(3-1.67)^2)*((1-2.33)^2+(2-2.33)^2+(4-2.33)^2))

[1] 3.527703

In [63]:
3.3333/3.5277

[1] 0.9448933

In [19]:
a = means.sparse(m.sparse[,450])

In [23]:
b = colMeans.sparse(m.sparse)
length(b)

[1] 1682

In [24]:
t(t(m.sparse) - b)*(m.sparse>0)

943 x 1682 sparse Matrix of class "dgCMatrix"
                                                                         
  [1,]  1.1216814 -0.2061069  0.96666667 -0.5502392 -0.3023256  1.4230769
  [2,]  0.1216814  .          .           .          .          .        
  [3,]  .          .          .           .          .          .        
  [4,]  .          .          .           .          .          .        
  [5,]  0.1216814 -0.2061069  .           .          .          .        
  [6,]  0.1216814  .          .           .          .          .        
  [7,]  .          .          .           1.4497608  .          .        
  [8,]  .          .          .           .          .          .        
  [9,]  .          .          .           .          .          1.4230769
 [10,]  0.1216814  .          .           0.4497608  .          .        
 [11,]  .          .          .           .          .          .        
 [12,]  .          .          .           1.4497608  .          . 